# RL Bot Analysis

Analyze and evaluate trained RL models from the rugs-rl-bot project.

**Prerequisites**: Set `RUGS_RL_BOT_PATH` in `jupyter/config.env`

In [ ]:
from _paths import *
import json
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from IPython.display import display, HTML

## 1. RL Bot Connection Check

In [ ]:
if RUGS_RL_BOT_PATH and RUGS_RL_BOT_PATH.exists():
    print(f"RL Bot connected: {RUGS_RL_BOT_PATH}")
    
    # Add to path for imports
    import sys
    if str(RUGS_RL_BOT_PATH) not in sys.path:
        sys.path.insert(0, str(RUGS_RL_BOT_PATH))
    
    # Check for models
    models_dir = RUGS_RL_BOT_PATH / "models"
    if models_dir.exists():
        models = list(models_dir.glob("**/*.zip")) + list(models_dir.glob("**/*.pkl"))
        print(f"Found {len(models)} model files")
    else:
        print("Models directory not found")
else:
    print("RL Bot not configured.")
    print("\nTo enable:")
    print("1. Edit jupyter/config.env")
    print("2. Set RUGS_RL_BOT_PATH=/path/to/rugs-rl-bot")
    print("3. Restart JupyterLab")

## 2. Load Sidebet Predictor

The SidebetPredictor provides real-time rug probability predictions:

In [ ]:
predictor = None

if RUGS_RL_BOT_PATH and RUGS_RL_BOT_PATH.exists():
    try:
        from rugs_bot.sidebet.predictor import SidebetPredictor
        
        predictor = SidebetPredictor()
        print("SidebetPredictor loaded successfully")
        print(f"\nModel info:")
        print(f"  Features: {predictor.feature_count if hasattr(predictor, 'feature_count') else 'N/A'}")
        
    except ImportError as e:
        print(f"Failed to import SidebetPredictor: {e}")
    except Exception as e:
        print(f"Error loading predictor: {e}")
else:
    print("RL Bot not configured - skipping predictor load")

## 3. Model Inventory

In [ ]:
if RUGS_RL_BOT_PATH and RUGS_RL_BOT_PATH.exists():
    models_dir = RUGS_RL_BOT_PATH / "models"
    
    if models_dir.exists():
        model_inventory = []
        
        for model_path in models_dir.glob("**/*"):
            if model_path.is_file() and model_path.suffix in ['.zip', '.pkl', '.pt', '.pth']:
                size_mb = model_path.stat().st_size / (1024 * 1024)
                model_inventory.append({
                    'name': model_path.name,
                    'type': model_path.suffix,
                    'size_mb': round(size_mb, 2),
                    'path': str(model_path.relative_to(models_dir))
                })
        
        if model_inventory:
            df_models = pd.DataFrame(model_inventory)
            df_models = df_models.sort_values('size_mb', ascending=False)
            print("Model Inventory:")
            display(df_models)
        else:
            print("No models found in models directory")
    else:
        print("Models directory does not exist")
else:
    print("RL Bot not configured")

## 4. Training Logs Analysis

Analyze training progress from TensorBoard logs:

In [ ]:
if RUGS_RL_BOT_PATH and RUGS_RL_BOT_PATH.exists():
    logs_dir = RUGS_RL_BOT_PATH / "logs"
    
    if logs_dir.exists():
        log_files = list(logs_dir.glob("**/*.json"))
        print(f"Found {len(log_files)} JSON log files")
        
        # Look for training progress files
        progress_files = list(logs_dir.glob("**/progress.json"))
        if progress_files:
            latest = max(progress_files, key=lambda p: p.stat().st_mtime)
            print(f"\nLatest training progress: {latest.name}")
            
            with open(latest) as f:
                progress = json.load(f)
            
            # Display key metrics
            if isinstance(progress, dict):
                print("\nTraining Metrics:")
                for key, value in list(progress.items())[:10]:
                    print(f"  {key}: {value}")
    else:
        print("Logs directory not found")
        print("Train a model first: python scripts/train_phase0_model.py")
else:
    print("RL Bot not configured")

## 5. Prediction Demo

Test the sidebet predictor with sample data:

In [ ]:
if predictor:
    # Simulate a game state observation
    sample_observation = {
        'price': 25.5,
        'tick': 50,
        'volume': 1000,
        'price_history': [20.0, 21.5, 23.0, 24.5, 25.5],
        'volatility': 0.15
    }
    
    try:
        # Get prediction
        prediction = predictor.predict(sample_observation)
        
        print("Sample Prediction:")
        print("=" * 40)
        print(f"Input: price={sample_observation['price']}, tick={sample_observation['tick']}")
        print(f"\nOutput:")
        for key, value in prediction.items():
            if isinstance(value, float):
                print(f"  {key}: {value:.4f}")
            else:
                print(f"  {key}: {value}")
    except Exception as e:
        print(f"Prediction failed: {e}")
        print("The predictor may require specific observation format.")
else:
    print("Predictor not loaded - see section 2")

## 6. Backtest Results

Load and analyze backtest results:

In [ ]:
if RUGS_RL_BOT_PATH and RUGS_RL_BOT_PATH.exists():
    results_dir = RUGS_RL_BOT_PATH / "results"
    
    if results_dir.exists():
        result_files = list(results_dir.glob("*.json"))
        print(f"Found {len(result_files)} result files")
        
        if result_files:
            latest = max(result_files, key=lambda p: p.stat().st_mtime)
            print(f"\nLatest: {latest.name}")
            
            with open(latest) as f:
                results = json.load(f)
            
            if isinstance(results, dict):
                print("\nResults Summary:")
                for key, value in list(results.items())[:15]:
                    if isinstance(value, float):
                        print(f"  {key}: {value:.4f}")
                    else:
                        print(f"  {key}: {value}")
    else:
        print("Results directory not found")
        print("Run evaluation: python scripts/evaluate_phase0_model.py")
else:
    print("RL Bot not configured")

## 7. Recording Statistics

Analyze game recordings used for training:

In [ ]:
recordings_dir = RUGS_RECORDINGS_DIR

if recordings_dir.exists():
    recordings = list(recordings_dir.glob("*.jsonl"))
    
    if recordings:
        total_size = sum(r.stat().st_size for r in recordings) / (1024 * 1024)
        
        print(f"Recording Statistics:")
        print("=" * 40)
        print(f"Total recordings: {len(recordings)}")
        print(f"Total size: {total_size:.1f} MB")
        print(f"Average size: {total_size / len(recordings):.2f} MB")
        
        # Sample a recording for event count
        sample = recordings[0]
        event_count = sum(1 for _ in open(sample))
        print(f"\nSample recording: {sample.name}")
        print(f"  Events: {event_count}")
    else:
        print("No recordings found")
else:
    print(f"Recordings directory not found: {recordings_dir}")
    print("Configure RUGS_RECORDINGS_DIR in config.env")

## 8. Performance Comparison

In [ ]:
# Known performance baselines from documentation
baselines = {
    'Random': {'win_rate': 16.7, 'roi': 0},
    'Sidebet v3': {'win_rate': 38.1, 'roi': 754},
    'Martingale': {'win_rate': 100, 'roi': 'N/A (never bankrupt)'}
}

print("Performance Baselines (from documentation):")
print("=" * 50)
for name, metrics in baselines.items():
    print(f"\n{name}:")
    print(f"  Win Rate: {metrics['win_rate']}%")
    print(f"  ROI: {metrics['roi']}%" if isinstance(metrics['roi'], (int, float)) else f"  ROI: {metrics['roi']}")

print("\n" + "=" * 50)
print("Target for RL Trading Bot:")
print("  Win Rate: 60%+")
print("  Rug Avoidance: 90%+")
print("  Bankruptcy Rate: <5%")

## Configuration Template

Example `jupyter/config.env` for RL Bot integration:

```bash
# RL Bot Integration
RUGS_RL_BOT_PATH=/home/nomad/Desktop/rugs-rl-bot

# Recording Data
RUGS_RECORDINGS_DIR=/home/nomad/rugs_recordings

# Other paths (optional)
VECTRA_PLAYER_PATH=
REPLAYER_PATH=/home/nomad/Desktop/REPLAYER
```

## Next Steps

- **01_event_explorer.ipynb** - Browse captured WebSocket events
- **02_canonical_review.ipynb** - Validate field promotions
- **03_coverage_dashboard.ipynb** - Documentation coverage